In [5]:
train = catalog.load("train_data")

2022-02-03 22:43:26,208 - kedro.io.data_catalog - INFO - Loading data from `train_data` (CSVDataSet)...


In [6]:
from sklearn.model_selection import GridSearchCV
from pmdarima.model_selection import RollingForecastCV
from time_series_kedro.pipelines.training.nodes._search import TSGridSearchCV
from time_series_kedro.extras.models import RandomForestForecaster

In [11]:
estimator = RandomForestForecaster(lags=12)
ts = train[train.family == train.family[0]].set_index("date").sales
param_grid = {"n_estimators": [10, 20, 30]}
cv = RollingForecastCV(step=365, h=30)
search = TSGridSearchCV(estimator, param_grid, cv_split=cv)
out = search.fit(ts)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   4 out of   9 | elapsed:    1.2s remaining:    1.5s
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed:    2.1s finished
/home/matheus/kedro_projects/time_series_kedro/venv/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/home/matheus/kedro_projects/time_series_kedro/venv/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/home/matheus/kedro_projects/time_series_kedro/venv/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index i

In [ ]:
sklearn.model_selection.GridSearchCV

In [ ]:
def ts_cv_split(
    ts: Union[pd.Series, np.array], 
    method: str = 'Rolling', 
    stride: int = 1, 
    frst: int =12, 
    pct_train: float = 0.7, 
    window_size: int = 24) -> Union[model_selection.RollingForecastCV, model_selection.SlidingWindowForecastCV]:

    
    if method == 'Rolling':
        
        initial = int(len(ts)*pct_train)
        n_trainings = len(ts) - (initial + frst)

        while (n_trainings < 3) & (frst != 1):
            frst -= 1
            n_trainings = len(ts) - (initial + frst)
            
        cv = model_selection.RollingForecastCV(step=stride, h=frst, initial=initial)
    
    elif method == 'SlidingWindow':
        cv = model_selection.SlidingWindowForecastCV(window_size=window_size, step=stride, h=frst)
        
    else: 
        raise NotImplementedError("The method you are trying to use is not implemented.")
    #logger.info(f"CV -> ts_size:{ts.shape[0]} initial:{initial} n_trainings:{n_trainings}")
    return cv

In [67]:
ts = train[train.family == train.family[0]].set_index("date").sales
model.fit(ts)
model.predict(12)

(1658, 13)
(1646, 12)


/home/matheus/kedro_projects/time_series_kedro/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([498.89889577, 348.34279797, 263.89981864, 260.97329327,
       275.60332837, 311.42060238, 427.2548127 , 470.65903338,
       353.96486698, 242.67743698, 221.165882  , 233.42478792])

In [64]:
model.fit(ts)
model.predict(12)

(1658, 13)
(1646, 12)


/home/matheus/kedro_projects/time_series_kedro/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([396.6663898 , 233.80268208, 174.46021891, 179.49602484,
       172.38989678, 198.61140083, 292.54027608, 255.68566003,
       125.88984773,  70.25013873,  76.12819303,  65.66434245])

ValueError: Expected 2D array, got 1D array instead:
array=[393. 317. 317. 308. 517. 534. 321. 302. 286. 331. 325. 460.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.